In [708]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [709]:
cars_origin = pd.read_pickle(os.path.join("data", "clean", "cars.pkl"))
cars_origin.head()

,Mã xe,URL,Tên hãng xe,Tên dòng xe,Năm sản xuất,Tỉnh,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Loại động cơ,Dung tích,Giá
0,5596529,https://bonbanh.com/xe-audi-a8-l-55-tfsi-quatt...,Audi,A8 L 55 TFSI Quattro,2020,TP HCM,Cũ,30000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,AWD,Xăng,3.0,2950.0
1,5555194,https://bonbanh.com/xe-audi-a6-2.0-tfsi-2014-5...,Audi,A6 2.0 TFSI,2014,Hà Nội,Cũ,94000,Nhập khẩu,Sedan,Số tự động,Xanh,Nâu,5,4,FWD,Xăng,2.0,560.0
2,5464113,https://bonbanh.com/xe-audi-a4-2.0-tfsi-2016-5...,Audi,A4 2.0 TFSI,2016,TP HCM,Cũ,50000,Nhập khẩu,Sedan,Số tự động,Đen,Đen,5,4,FWD,Xăng,2.0,690.0
3,5532313,https://bonbanh.com/xe-audi-q3-2.0-quattro-201...,Audi,Q3 2.0 Quattro,2016,Hà Nội,Cũ,76000,Nhập khẩu,Crossover,Số tự động,Xanh,Đen,5,5,AWD,Xăng,2.0,699.0
4,5524426,https://bonbanh.com/xe-audi-q5-2.0-at-2016-552...,Audi,Q5 2.0 AT,2016,Hà Nội,Cũ,90000,Nhập khẩu,SUV,Số tự động,Trắng,Kem,5,5,AWD,Xăng,2.0,739.0


In [710]:
cars_origin["Tên hãng xe"] = cars_origin["Tên hãng xe"].apply(lambda x: x.replace(' ', '').strip().lower())

### Drop columns

In [711]:
cars = cars_origin.drop(
    columns=["Mã xe", "Tên dòng xe", "Tình trạng", "URL"], axis=1
)

### Converting all non-numeric features into numeric features.

In [712]:
cars.dtypes

Tên hãng xe        object
Năm sản xuất        int64
Tỉnh               object
Số Km đã đi         int64
Xuất xứ            object
Kiểu dáng          object
Hộp số             object
Màu ngoại thất     object
Màu nội thất       object
Số chỗ ngồi         int64
Số cửa              int64
Dẫn động           object
Loại động cơ       object
Dung tích         float64
Giá               float64
dtype: object

In [713]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding
non_numeric_cols = [
    "Tên hãng xe", "Năm sản xuất", "Tỉnh", "Xuất xứ",
    "Kiểu dáng", "Hộp số", "Màu ngoại thất",
    "Màu nội thất", "Dẫn động", "Loại động cơ"
]

encoders = {}

for col in non_numeric_cols:
    le = LabelEncoder()
    cars[col] = le.fit_transform(cars[col])

    encoders[col] = le

In [714]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Standard Scaler
cols = ["Số Km đã đi"]
scalers = {}

for col in cols:
    scaler = StandardScaler()
    cars[col] = scaler.fit_transform(cars[[col]])

    scalers[col] = scaler

In [715]:
cars.head()

,Tên hãng xe,Năm sản xuất,Tỉnh,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Loại động cơ,Dung tích,Giá
0,0,31,45,-0.008785,1,6,1,15,15,5,4,1,2,3.0,2950.0
1,0,25,17,-0.006379,1,6,1,13,9,5,4,2,2,2.0,560.0
2,0,27,45,-0.008033,1,6,1,15,15,5,4,2,2,2.0,690.0
3,0,27,17,-0.007056,1,3,1,13,15,5,5,1,2,2.0,699.0
4,0,27,17,-0.006530,1,5,1,10,6,5,5,1,2,2.0,739.0


In [716]:
X = cars.iloc[:, :-1].values
y = cars.iloc[:, -1].values
y = y.reshape(-1, 1)
len(X), len(y)

(26406, 26406)

In [717]:
X, y

(array([[ 0., 31., 45., ...,  1.,  2.,  3.],
        [ 0., 25., 17., ...,  2.,  2.,  2.],
        [ 0., 27., 45., ...,  2.,  2.,  2.],
        ...,
        [26., 33., 17., ...,  1.,  1.,  2.],
        [26., 32., 17., ...,  1.,  2.,  2.],
        [26., 26., 17., ...,  1.,  2.,  2.]]),
 array([[2950.],
        [ 560.],
        [ 690.],
        ...,
        [4189.],
        [1739.],
        [1450.]]))

## Linear Regression

In [718]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [719]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
def evaluate(y_pred, y_test):
    np.set_printoptions(precision=0)
    values = np.concatenate((y_pred.reshape(-1, 1), y_test.reshape(-1, 1)), 1)
    print(values)
    # Evaluate here
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"R^2 Score: {r2}")
    print(f"Mean Absolute Percentage Error: {mape}")
    return mse, r2, mae, mape

In [720]:
# No Apply Standard Scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

mse, r2, mae, mape = evaluate(y_pred, y_test)

[[1869.  869.]
 [1525. 3150.]
 [ 326.  220.]
 ...
 [  47.  336.]
 [3418. 2520.]
 [ 986. 1219.]]
Mean Squared Error: 1528919.685373685
Mean Absolute Error: 779.9060207958382
R^2 Score: 0.4762413097744177
Mean Absolute Percentage Error: 1.1852430648920134


In [721]:
# Apply Standard Scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc_X = StandardScaler()
sc_y = StandardScaler()

X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)
X_test = sc_X.transform(X_test)
y_test = sc_y.transform(y_test)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
y_pred = sc_y.inverse_transform(y_pred)
y_test = sc_y.inverse_transform(y_test)

mse, r2, mae, mape = evaluate(y_pred, y_test)

[[1869.  869.]
 [1525. 3150.]
 [ 326.  220.]
 ...
 [  47.  336.]
 [3418. 2520.]
 [ 986. 1219.]]
Mean Squared Error: 1528919.685373688
Mean Absolute Error: 779.9060207958419
R^2 Score: 0.4762413097744168
Mean Absolute Percentage Error: 1.1852430648920191


In [722]:
# Apply MinMax Scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc = MinMaxScaler()
y_train = sc.fit_transform(y_train.reshape(-1, 1))
y_test = sc.transform(y_test.reshape(-1, 1))

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
y_pred = sc.inverse_transform(y_pred)
y_test = sc.inverse_transform(y_test)

mse, r2, mae, mape = evaluate(y_pred, y_test)

[[1869.  869.]
 [1525. 3150.]
 [ 326.  220.]
 ...
 [  47.  336.]
 [3418. 2520.]
 [ 986. 1219.]]
Mean Squared Error: 1528919.6853736849
Mean Absolute Error: 779.9060207958379
R^2 Score: 0.4762413097744177
Mean Absolute Percentage Error: 1.1852430648920123


## Polynomial Linear Regression

In [723]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [724]:
from sklearn.preprocessing import PolynomialFeatures
poly_feature = PolynomialFeatures(degree=2)
X_poly = poly_feature.fit_transform(X_train)
X_test = poly_feature.transform(X_test)
X_poly.shape

(21124, 120)

In [725]:
regressor = LinearRegression()
regressor.fit(X_poly, y_train)

y_pred = regressor.predict(X_test)
_ = evaluate(y_pred, y_test)

[[ -465.   141.]
 [ 1064.   679.]
 [ 2300.  3099.]
 ...
 [ 1093.  1145.]
 [  370.   485.]
 [-1157.   285.]]
Mean Squared Error: 4462780.429599628
Mean Absolute Error: 1241.2211813872796
R^2 Score: -0.7365906002564517
Mean Absolute Percentage Error: 2.184560393156465


## Support Vector Regression

In [726]:
from sklearn.svm import SVR

In [727]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [728]:
regressor = SVR(kernel="rbf")
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
_ = evaluate(y_pred, y_test)

/home/admin/miniconda3/envs/ml-env/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 627. 1850.]
 [ 743. 1435.]
 [ 776. 1125.]
 ...
 [ 612.  290.]
 [ 777.  685.]
 [ 766. 3650.]]
Mean Squared Error: 568639331.1577777
Mean Absolute Error: 998.5162104853462
R^2 Score: -0.0011956177425220815
Mean Absolute Percentage Error: 0.5114555976927123


In [729]:
sc_X = StandardScaler()
sc_y = StandardScaler()

X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)
X_test = sc_X.transform(X_test)
y_test = sc_y.transform(y_test)

regressor = SVR(kernel="rbf")
regressor.fit(X_train, y_train)

/home/admin/miniconda3/envs/ml-env/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [730]:
y_pred = regressor.predict(X_test)
# Eval before inverse transform
_ = evaluate(y_pred, y_test)

[[ 1.  0.]
 [ 0.  0.]
 [-0. -0.]
 ...
 [-1. -1.]
 [-0. -0.]
 [ 3.  1.]]
Mean Squared Error: 214.62720701591866
Mean Absolute Error: 0.32225716780606306
R^2 Score: 0.0041832273600976855
Mean Absolute Percentage Error: 2.260090321970516


In [731]:
# Eval after inverse transform
y_pred = sc_y.inverse_transform(y_pred.reshape(-1, 1))
y_test = sc_y.inverse_transform(y_test.reshape(-1, 1))
_ = evaluate(y_pred, y_test)

[[2190. 1850.]
 [1802. 1435.]
 [1070. 1125.]
 ...
 [ 306.  290.]
 [ 837.  685.]
 [5628. 3650.]]
Mean Squared Error: 565584360.8529222
Mean Absolute Error: 523.1291191073032
R^2 Score: 0.004183227360097463
Mean Absolute Percentage Error: 0.2005831884668667


## Decision Tree Regression

In [748]:
from sklearn.tree import DecisionTreeRegressor

In [749]:
X_tree = cars_origin.drop(columns=["Mã xe", "URL", "Giá", "Tên dòng xe"])
y_tree = cars_origin["Giá"]

In [750]:
X_tree_train, X_tree_test, y_tree_train, y_tree_test = train_test_split(X_tree, y_tree)

In [751]:
X_tree_train

,Tên hãng xe,Năm sản xuất,Tỉnh,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Hộp số,Màu ngoại thất,Màu nội thất,Số chỗ ngồi,Số cửa,Dẫn động,Loại động cơ,Dung tích
575,bmw,2020,Hà Nội,Cũ,18000,Nhập khẩu,SUV,Số tự động,Đen,Nâu,7,5,AWD,Xăng,3.0
16435,mercedesbenz,2022,Hà Nội,Cũ,10000,Lắp ráp trong nước,Sedan,Số tự động,Trắng,Đen,5,4,RFD,Xăng,2.0
9928,kia,2011,Hà Nội,Cũ,125280,Nhập khẩu,Sedan,Số tự động,Bạc,Đen,5,4,FWD,Xăng,1.6
17543,mitsubishi,2024,Hà Nội,Mới,0,Nhập khẩu,SUV,Số tự động,Đen,Đen,7,5,FWD,Xăng,1.5
19917,suzuki,2019,TP HCM,Cũ,52000,Nhập khẩu,Hatchback,Số tự động,Đỏ,Đen,5,5,FWD,Xăng,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16190,mercedesbenz,2021,Hà Nội,Cũ,51488,Nhập khẩu,Sedan,Số tự động,Nâu,Kem,5,4,RFD,Xăng,3.0
19657,porsche,2022,TP HCM,Cũ,12500,Nhập khẩu,SUV,Số tự động,Trắng,Xám,5,5,AWD,Xăng,3.0
22280,toyota,2007,Hà Nội,Cũ,80000,Lắp ráp trong nước,Crossover,Số tay,Xanh,Kem,8,5,RFD,Xăng,2.0
22852,toyota,2024,TP HCM,Mới,0,Nhập khẩu,Crossover,Số tự động,Trắng,Nhiều màu,8,5,FWD,Xăng,2.0


In [752]:
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_tree_train.values.reshape(-1, 1), y_tree_train.values.reshape(-1, 1))

ValueError: could not convert string to float: 'bmw'

## Random Forest Regression